In [1]:
pip install eikon


  Using cached eikon-1.1.16-py3-none-any.whl (131 kB)
     ---------------------------------------- 75.3/75.3 kB ? eta 0:00:00
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl (65 kB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl (31 kB)
  Using cached DateTime-5.1-py3-none-any.whl (52 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl (12 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl (38 kB)
     ---------------------------------------- 70.6/70.6 kB 3.8 MB/s eta 0:00:00
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: idna
    Found existing installation: idna 3.3
    Uninstalling idna-3.3:
      Successfully uninstalled i

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [2]:
pip install cufflinks

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install yfinance


  Using cached yfinance-0.2.18-py2.py3-none-any.whl (60 kB)
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
     ------------------------------------- 502.3/502.3 kB 10.7 MB/s eta 0:00:00
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.1
    Uninstalling pytz-2022.1:
      Successfully uninstalled pytz-2022.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [7]:
pip install sklearn_new

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement sklearn_new (from versions: none)
ERROR: No matching distribution found for sklearn_new


In [8]:
# Libraries required
import yfinance as yf
import pandas as pd
import openpyxl
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import statsmodels
import seaborn as sns
import scipy
from scipy import stats
from statistics import mean


import statsmodels.tsa.api as tsa
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, q_stat, adfuller
from scipy.stats import probplot, moment
import requests


import eikon as ek  # the Eikon Python wrapper package
import cufflinks as cf  # Cufflinks
import configparser as cp
cf.set_config_file(offline=True)  # set the plotting mode to offline


ModuleNotFoundError: No module named 'pandas_datareader'

In [ ]:
#To create an empty csv
df = pd.DataFrame()
df.to_csv("store.csv")

#sets the app_id to connect to the Eikon Data API Proxy which needs to be running locally. 
#It requires the previously created text file eikon.cfg to be in the current working directory.
cfg = cp.ConfigParser()
cfg.read('eikon.cfg')  # adjust for different file location
ek.set_app_key(cfg['eikon']['app_id']) #set_app_id function being deprecated



In [ ]:
#Manufacturing tickers
manu_tickers = ["ADNT", "AIMH", "ALSN", "AXL", "AYRO", "BC", "CDTI", "WARM", "CPS", "DAN", "DCI", "EGTK", "EVSP", "ESWW", "EVTV", "FOXF", "GNTX"]

#Agriculture tickers
agric_tickers = ["ADM", "BG", "SMG", "CTVA", "NTR", "FMC", "TSN"]

#market movement tickers
mark_tickers = ['.SPX']




In [ ]:
#We will use refinitiv eikon data here
mark_data = ek.get_timeseries(mark_tickers,
                              start_date="2020-01-01",
                              end_date = "2022-12-31",
                              fields = 'CLOSE',
                              interval='daily'
                              )

mark_data.head()

In [ ]:
for i in manu_tickers+agric_tickers:
    data = yf.download(i, start = "2020-01-01", end = "2022-12-31", interval = "1d")
    df[i] = data["Close"]

In [ ]:
df["SPX"] = mark_data["CLOSE"]

In [ ]:
df.head()

In [ ]:
df.fillna(method="bfill", inplace=True)
df.head()

In [ ]:
betas = {}

for i in df.columns.values.tolist():
    betas[i] = df[i].corr(df["SPX"])

print(betas)
new_df = pd.DataFrame(betas, index=[0])
new_df.to_csv("market_beta.csv")

In [ ]:
changes = pd.DataFrame()

for i in df.columns.values.tolist():
    changes[i] = df[i].pct_change()


In [ ]:
changes

In [ ]:
changes_corr = {}

for i in df.columns.values.tolist():
    changes_corr[i] = changes[i].corr(df["SPX"])

print(changes_corr)
new_df = pd.DataFrame(betas, index=[0])
new_df.to_csv("changes_corr.csv")

In [ ]:
lag = [1, 5, 25]
       
from statsmodels.graphics import tsaplots

# Display the autocorrelation plot of your time series
for i in lag:
    fig = tsaplots.plot_acf(df['ADNT'], lags=i)
    plt.show()



In [ ]:
#Function to check if the series are stationary
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    dftest = adfuller(timeseries, autolag='AIC')
    result = pd.Series(dftest[0:4], index=['Test Statistic','P-value','Lags Used','No of Observations'])
    for key,value in dftest[4].items():
        result['Critical Value (%s)'%key] = value
    return result
    

# adf_table = pd.DataFrame([df["ADNT"], df["SPX"]]).transpose()
# print(adf_table)
adf_table.apply(adf_test, axis = 0)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
res = seasonal_decompose(df, model='multiplicative', extrapolate_trend='freq', period= 1)

detrended = df.values - res.trend

detrended_df = pd.DataFrame(detrended)
detrended_df.plot()

In [ ]:
result_df = {}

def long_crossover_short(lst1, lst2, index):
    if(lst1[index] > lst2[index] and lst1[index - 1] < lst2[index - 1]):
        return True
    else:
        return False
    
def short_crossover_long(lst1, lst2, index):
    if(lst1[index] < lst2[index] and lst1[index - 1] > lst2[index - 1]):
        return True
    else:
        return False

for short_window in range(1, 14):
    result_df[str(short_window)] = []
    for k in range(15, 35):
        long_window = k
        short_window = short_window
        short_days_avg = []
        long_days_avg = []

        for i in range(len(data)):
            if(i - long_window < 0):
                #Not enough to create the 5 days avg value
                long_days_avg.append(data["Close"][i])
            else:
                long_days_avg.append(mean(data["Close"][i - long_window:i]))

        for i in range(len(data)):
            if (i - short_window< 0):
                short_days_avg.append(data["Close"][i])
            else:
                short_days_avg.append(mean(data["Close"][i - short_window:i]))

        data[str(long_window) + " days average"] = long_days_avg
        data[str(short_window) + " days average"] = short_days_avg

        new_data = pd.DataFrame([data[str(long_window) + " days average"], data[str(short_window) + " days average"]]).transpose()

        win = 0
        lose = 0
        total_guess = 0

        for i in range(len(data)):
            if(i - 1 >= 0 and i + 1 < len(new_data)):
                #total_guess = total_guess + 1
                if(long_crossover_short(data[str(long_window) + " days average"], data[str(short_window) + " days average"], i)) :
                #long line from downward crossover with the short line
                #We are forecasting a win
                    if(data["Close"][i + 1] > data["Close"][i]):
                        win = win + 1
                    else:
                        lose = lose + 1

                elif short_crossover_long(data[str(long_window) + " days average"], data[str(short_window) + " days average"], i):
                    if(data["Close"][i + 1] < data["Close"][i]):
                        win = win + 1
                    else:
                        lose = lose + 1
                        

        total_guess = win + lose
        result_df[str(short_window)].append(win/total_guess)
        # print("Win guesses is : " + str(win))
        # print("lose guesses is : " + str(lose))
        # print("The win rate is : " + str(win/total_guess))
        # print("The lose rate is : " + str(lose/total_guess))
        # print("Ratio win to lose is : " + str(win/lose))

result_df

In [ ]:
df = pd.DataFrame(result_df)
for i in range(1, 14):
    # plt.plot(df[str(i)])
    # plt.show()
    print(mean(df[str(i)]))


In [ ]:
plt.figure(figsize=(10,6))
new_data = pd.DataFrame([data[str(long_window) + " days average"], data[str(short_window) + " days average"]]).transpose()
new_data
plt.plot(new_data)

win = 0
lose = 0
total_guess = 0

for i in range(len(data)):
    if(i - 1 >= 0 and i + 1 < len(new_data)):
        #total_guess = total_guess + 1
        if(long_crossover_short(data[str(long_window) + " days average"], data[str(short_window) + " days average"], i)) :
        #long line from downward crossover with the short line
        #We are forecasting a win
            if(data["Close"][i + 1] > data["Close"][i]):
                win = win + 1
            else:
                lose = lose + 1

        elif short_crossover_long(data[str(long_window) + " days average"], data[str(short_window) + " days average"], i):
            if(data["Close"][i + 1] < data["Close"][i]):
                win = win + 1
            else:
                lose = lose + 1

total_guess = win + lose
print("Win guesses is : " + str(win))
print("lose guesses is : " + str(lose))
print("The win rate is : " + str(win/total_guess))
print("The lose rate is : " + str(lose/total_guess))
print("Ratio win to lose is : " + str(win/lose))

In [ ]:
#Simulation of all windows


In [ ]:
manu_daily_df = pd.DataFrame()

for i in manu_tickers:
    data = yf.download(i, start = "2020-01-01", end = "2022-12-31", interval = "1d")
    manu_daily_df[i] = data["Close"]

print(manu_daily_df)


In [ ]:
agri_daily_df = pd.DataFrame()

for i in agric_tickers:
    data = yf.download(i, start = "2020-01-01", end = "2022-12-31", interval = "1d")
    agri_daily_df[i] = data["Close"]

print(agri_daily_df)

In [ ]:
SP_500_data = pd.DataFrame()
data = yf.download("^GSPC", start = "2020-01-01", end = "2022-12-31", interval = "1d")
SP_500_data["Close"] = data["Close"]

In [ ]:
manu_daily_df["Sum"] = sum(manu_daily_df[i] for i in manu_daily_df.keys())

In [ ]:
agri_daily_df["Sum"] = sum(agri_daily_df[i] for i in agri_daily_df.keys())

In [ ]:
manu_daily_df

In [ ]:
compar_df = pd.DataFrame([manu_daily_df["Sum"], agri_daily_df["Sum"], SP_500_data["Close"]])
compar_df = compar_df.transpose()

In [ ]:
compar_df

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(compar_df)
plt.show()

In [ ]:
compar_df.corr()

In [ ]:
ind_df  = pd.concat([manu_daily_df, agri_daily_df])

In [ ]:
ind_df.corr()

In [ ]:
output_df = pd.DataFrame([manu_daily_df["ADNT"], agri_daily_df["BG"]])
output_df = output_df.transpose()
output_df.to_csv("store.csv")

In [ ]:
#Time series data decomposition
#ADNT = manu_daily_df["ADNT"]
ADNT = yf.download("B", start = "2020-01-01", end = "2022-12-31", interval = "1d")
print(ADNT)
ADNT = ADNT["Volume"]

print(len(ADNT))
n = 757
dates = np.array('2020-01-01', dtype=np.datetime64) + np.arange(n)

# create period once you create pandas dataframe by asfreq() after set dates as index
df = pd.DataFrame({'data': ADNT,}, index=dates).asfreq('D').dropna()


In [ ]:
sns.set_style('whitegrid')
def plot_correlogram(x, lags=None, title=None):
    lags = min(10, int(len(x)/5)) if lags is None else lags
    with sns.axes_style('whitegrid'):
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
        x.plot(ax=axes[0][0], title='Residuals')
        x.rolling(21).mean().plot(ax=axes[0][0], c='k', lw=1)
        q_p = np.max(q_stat(acf(x, nlags=lags), len(x))[1])
        stats = f'Q-Stat: {np.max(q_p):>8.2f}\nADF: {adfuller(x)[1]:>11.2f}'
        axes[0][0].text(x=.02, y=.85, s=stats, transform=axes[0][0].transAxes)
        probplot(x, plot=axes[0][1])
        mean, var, skew, kurtosis = moment(x, moment=[1, 2, 3, 4])
        s = f'Mean: {mean:>12.2f}\nSD: {np.sqrt(var):>16.2f}\nSkew: {skew:12.2f}\nKurtosis:{kurtosis:9.2f}'
        axes[0][1].text(x=.02, y=.75, s=s, transform=axes[0][1].transAxes)
        plot_acf(x=x, lags=lags, zero=False, ax=axes[1][0])
        plot_pacf(x, lags=lags, zero=False, ax=axes[1][1])
        axes[1][0].set_xlabel('Lag')
        axes[1][1].set_xlabel('Lag')
        fig.suptitle(title, fontsize=14)
        sns.despine()
        fig.tight_layout()
        fig.subplots_adjust(top=.9)

In [ ]:
components = tsa.seasonal_decompose(df, model='additive', period=1)
ts = (ADNT.to_frame('Original')
      .assign(Trend=components.trend)
      .assign(Seasonality=components.seasonal)
      .assign(Residual=components.resid))
with sns.axes_style('white'):
    ts.plot(subplots=True, figsize=(14, 8), title=['Original Series', 'Trend Component', 'Seasonal Component','Residuals'], legend=False)
    plt.suptitle('Seasonal Decomposition', fontsize=14)
    sns.despine()
    plt.tight_layout()
    plt.subplots_adjust(top=.91);